# Model Hyperparameter Search

Afonso Lavado | 20220631@novaims.unl.pt

Afonso Reyna | 20191197@novaims.unl.pt

Fernando Cruz | 20220646@novaims.unl.pt

Mariana Rodrigues | 20220627@novaims.unl.pt

Pedro Fernandes | 20220592@novaims.unl.pt 

Data Source | https://drive.google.com/file/d/1bs3GM_j9XScOEm-mNOG3ZEYg7-NVTc1C/view?usp=share_link

In this Jupyter notebook, we will be exploring the process of hyperparameter tuning using the Keras Tuner package to improve the performance of a deep learning model. We will be building upon the work we previously did, where we developed several deep learning models for the image dataset at hand and identified the best performing model based on evaluation metrics.

Hyperparameter tuning is the process of searching for the optimal combination of hyperparameters that will result in the best model performance. Keras Tuner is a powerful Python library that provides a framework for performing this search. By using Keras Tuner, we can automate the process of trying out different combinations of hyperparameters and quickly identify the combination that results in the best model performance.

In this notebook, we will first load and preprocess the data that we will be using for our task. Then, we will select the best performing model from our previous work and define its architecture. After that, we will use Keras Tuner to search for the optimal hyperparameters for this model. Finally, we will evaluate the performance of the tuned model using performance measures (precision / recall / accuracy / f1 score) as well as the confusion matrix.

## 1 - Imports / Preprocess

In [ ]:
pip install keras_tuner

In [4]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing import image_dataset_from_directory

import numpy as np
import pandas as pd

import zipfile
import time
import shutil
import os
import gdown
import random

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras_tuner as kt

#extra
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers
from tensorflow.keras.preprocessing import image_dataset_from_directory, image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import InceptionV3
from keras import regularizers

In [ ]:
#The next two cells are distinct methods of downloading the data, choose only 1

#Downloading the Dataset directly from google drive link (prevents if done several times)
!gdown --fuzzy https://drive.google.com/file/d/1bs3GM_j9XScOEm-mNOG3ZEYg7-NVTc1C/view?usp=share_link

#Extracting the Dataset to Content
zip_ = zipfile.ZipFile('Sports10_clean.zip')
zip_.extractall()
zip_.close()

In [5]:
#Download the dataset from personal drive folder 
from google.colab import drive
drive.mount('/content/gdrive')

!unzip -q gdrive/My\Drive/DL/Sports10_clean.zip

Mounted at /content/gdrive


In [6]:
#Defining the path to the folder
path = "/content/content/"
main_folder, training_folder, testing_folder = "Sports10_clean/", "training_set/", "test_set/"

In [7]:
image_size=(128, 128)
crop_to_aspect_ratio=True
color_mode='rgb'
batch_size= 256
label_mode="categorical"
validation_split=0.3
shuffle=True
seed=0
num_classes = 10

In [8]:
rmdir /content/content/Sports10_clean/test_set/".ipynb_checkpoints"

In [9]:
rmdir /content/content/Sports10_clean/training_set/".ipynb_checkpoints"

In [10]:
# Generate an object of type tf.data.Dataset 
ds_train, ds_val = image_dataset_from_directory(path + main_folder + training_folder, 
                                                image_size=image_size,
                                                crop_to_aspect_ratio=crop_to_aspect_ratio,
                                                color_mode=color_mode,
                                                batch_size=batch_size,
                                                labels = "inferred",
                                                label_mode=label_mode, # categorical
                                                subset='both',
                                                validation_split=validation_split, 
                                                shuffle=shuffle,
                                                seed=seed
                                                )
# Check object properties
print("\nObject's type:\t", type(ds_train))
print("Is it a tf.data.Dataset?\t R:",isinstance(ds_train, tf.data.Dataset))
print("Classes:", ds_train.class_names)

Found 72552 files belonging to 10 classes.
Using 50787 files for training.
Using 21765 files for validation.

Object's type:	 <class 'tensorflow.python.data.ops.batch_op._BatchDataset'>
Is it a tf.data.Dataset?	 R: True
Classes: ['AmericanFootball', 'Basketball', 'BikeRacing', 'CarRacing', 'Fighting', 'Hockey', 'Soccer', 'TableTennis', 'Tennis', 'Volleyball']


In [11]:
#Generate test dataset
ds_test = image_dataset_from_directory(path + main_folder + testing_folder, 
                                       image_size=image_size,
                                       crop_to_aspect_ratio=crop_to_aspect_ratio,
                                       color_mode=color_mode,
                                       batch_size=batch_size,
                                       label_mode=label_mode,                                     
                                       shuffle=shuffle,
                                       seed=seed)

Found 27448 files belonging to 10 classes.


## 2 - Modeling

In [ ]:
# # This is the model that we want to apply hyperparameter optimization to

# preprocess = Sequential([layers.Rescaling(1./255)], name="preprocessing")

# cnn_winner = Sequential([
#                    preprocess,
#                    layers.Conv2D(16, (3, 3), activation='sigmoid', input_shape=(128, 128, 3), name='conv1'),
#                    layers.BatchNormalization(),
#                    layers.MaxPooling2D((2, 2)),
                   
#                    layers.Conv2D(32, (3, 3), activation='sigmoid', name='conv2'),
#                    layers.BatchNormalization(),
#                    layers.MaxPooling2D((2, 2)),
                   
#                    layers.Conv2D(64, (3, 3), activation='sigmoid', name='conv3'),
#                    layers.BatchNormalization(),
#                    layers.MaxPooling2D((2, 2)),
                   
#                    layers.Conv2D(128, (3, 3), activation='sigmoid', name='conv4'),
#                    layers.BatchNormalization(),
#                    layers.MaxPooling2D((2, 2)),
                   
#                    layers.Flatten(),
                   
#                    layers.Dropout(0.2),
                   
#                    layers.Dense(128, activation='sigmoid', name='dense1'),
#                    layers.Dropout(0.2),
                   
#                    layers.Dense(64, activation='sigmoid', name='dense2'),
                   
#                    layers.Dense(10, activation='softmax', name='output')
#                    ],
#                   name = 'CNN-Winner')

# cnn_winner.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

---

The `Build_Model()` function below creates the pipeline to apply hyperparameter tuning to the optimal model above. It takes as argument the function `hp` that allows to create a range of values through which the tuner will iterate over.

---

In [ ]:
def build_model(hp):
    preprocess = Sequential([layers.Rescaling(1./255)], name="preprocessing")
    # Initialize sequential API and start building model.
    model = keras.Sequential()
    model.add(preprocess)

    #input Layer
    model.add(layers.Conv2D(16, (3, 3), activation='sigmoid', input_shape=(128, 128, 3), name='conv1')),
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    #Conv 2D layers
    model.add(layers.Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=64, step=16), 
                            kernel_size = (3, 3), activation='sigmoid', input_shape=(128, 128, 3), 
                            name='conv2'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(filters=hp.Int('conv_2_filter', min_value=64, max_value=128, step=16), 
                            kernel_size = (3, 3), activation='sigmoid', input_shape=(128, 128, 3), 
                            name='conv3'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(filters=hp.Int('conv_3_filter', min_value=128, max_value=256, step=16), 
                            kernel_size = (3, 3), activation='sigmoid', input_shape=(128, 128, 3), 
                            name='conv4'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    #Flatten
    model.add(layers.Flatten())
    model.add(layers.Dropout(hp.Float("dropout_1" , 0, 0.3, step=0.1)))

    #Dense
    model.add(layers.Dense(units=hp.Int("units_1", min_value=32, max_value=512, step=32), 
                           activation='sigmoid', name='dense1'))
    model.add(layers.Dropout(hp.Float("dropout_2", 0, 0.3, step=0.1)))

    model.add(layers.Dense(units=hp.Int("units_2", min_value=32, max_value=512, step=32), 
                           activation='sigmoid', name='dense2'))

    #Output 
    model.add(layers.Dense(10, activation='softmax', name='output'))
    
    # Define optimizer, loss, and metrics
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

## 3 - Tuning

In [ ]:
# Instantiate the tuner
tuner = kt.RandomSearch(build_model,
                          objective= kt.Objective("val_accuracy", direction="max"),
                          directory="Keras Tuner",
                          overwrite=True,                        
                          executions_per_trial=1, 
                          max_trials=8,
                          seed=seed)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 7
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
conv_2_filter (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
conv_3_filter (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 256, 'step': 16, 'sampling': 'linear'}
dropout_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
dropout_2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}


---

The search method below is very computationally expensive, as it iterates through the epochs over 8 max trials in each. Therefore, we opted for 5 epochs as the accuracy converged to satisfying numbers very early (close to 0.99 in validation)

---



In [ ]:
tuner.search(ds_train, epochs = 5, validation_data = ds_val)

Trial 5 Complete [00h 03m 45s]
val_accuracy: 0.9936595559120178

Best val_accuracy So Far: 0.9960946440696716
Total elapsed time: 00h 18m 51s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
32                |32                |conv_1_filter
112               |96                |conv_2_filter
176               |224               |conv_3_filter
0                 |0.1               |dropout_1
32                |224               |units_1
0                 |0.2               |dropout_2
288               |64                |units_2

Epoch 1/5
199/199 [==============================] - 48s 215ms/step - loss: 0.8767 - accuracy: 0.8405 - val_loss: 3.1045 - val_accuracy: 0.1028
Epoch 2/5
199/199 [==============================] - 44s 217ms/step - loss: 0.0829 - accuracy: 0.9895 - val_loss: 1.1425 - val_accuracy: 0.6526
Epoch 3/5
199/199 [==============================] - 44s 214ms/step - loss: 0.0344 - accuracy: 0.9945 - val_loss: 0.0990 - val_accuracy: 0.9728
E

KeyboardInterrupt: ignored

**`NOTE`** The tunning process was interrupted manually at trial #6 due to satisfactory results and high running time/RAM usage, which is the origin of the error displayed

In [ ]:
# # These are the best hyperparameters obatined
# Best Value        |Hyperparameter
# 32                |conv_1_filter
# 96                |conv_2_filter
# 224               |conv_3_filter
# 0.1               |dropout_1
# 224               |units_1
# 0.2               |dropout_2
# 64                |units_2

In [ ]:
#Stopping function for training the models
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=2, restore_best_weights=True)

# Train the model that was output from the tuner (with the optimal hyperparameters)
winner_model_hp = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

history_winner = winner_model_hp.fit(ds_train, epochs=10, validation_data= ds_val, callbacks = [es])

Epoch 1/10
199/199 [==============================] - 48s 215ms/step - loss: 0.0169 - accuracy: 0.9963 - val_loss: 0.0172 - val_accuracy: 0.9953
Epoch 2/10
199/199 [==============================] - 44s 217ms/step - loss: 0.0075 - accuracy: 0.9985 - val_loss: 0.0141 - val_accuracy: 0.9963
Epoch 3/10
199/199 [==============================] - 44s 216ms/step - loss: 0.0042 - accuracy: 0.9994 - val_loss: 0.0078 - val_accuracy: 0.9980
Epoch 4/10
199/199 [==============================] - 45s 219ms/step - loss: 0.0032 - accuracy: 0.9994 - val_loss: 0.0317 - val_accuracy: 0.9906
Epoch 5/10
199/199 [==============================] - 45s 217ms/step - loss: 0.0192 - accuracy: 0.9951 - val_loss: 0.0406 - val_accuracy: 0.9883


In [ ]:
#Obtain the score on the test dataset
score = winner_model_hp.evaluate(ds_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.0866353511810303
Test accuracy: 0.7639901041984558


In [ ]:
# Export the model
#winner_model_hp.save("winner_model_hp.h5")

In [ ]:
# Load a model externally
#model = tf.keras.saving.load_model("/content/winner_model_hp.h5")

In [ ]:
# score = model.evaluate(ds_test, verbose=1)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

108/108 [==============================] - 129s 1s/step - loss: 1.0867 - accuracy: 0.7640
Test loss: 1.0866645574569702
Test accuracy: 0.763953685760498


In [12]:
# #Downloading the tuned winner model from google drive
# !gdown --fuzzy https://drive.google.com/file/d/1lolhRD0s1Ns55w3pOp2AfXCMJ_PhQPCJ/view?usp=share_link

# # Load the winner model from the local storage
# model = tf.keras.saving.load_model("/content/winner_model_hp.h5")

Downloading...
From: https://drive.google.com/uc?id=1lolhRD0s1Ns55w3pOp2AfXCMJ_PhQPCJ
To: /content/winner_model_hp.h5
100% 24.7M/24.7M [00:00<00:00, 143MB/s]


## 4 - Performance Metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix


# Get the true labels and predicted labels for the test dataset
true_labels = []
pred_labels = []
for images, labels in ds_test:
    predictions = model.predict(images)
    pred_labels += list(np.argmax(predictions, axis=1))
    true_labels += list(np.argmax(labels.numpy(), axis=1))

In [23]:
# Calculate accuracy
accuracy = accuracy_score(true_labels, pred_labels)

# Calculate precision, recall, and f1-score
precision = precision_score(true_labels, pred_labels, average='macro')
recall = recall_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')

# Print the evaluation metrics
print('Accuracy:', round(accuracy,4))
print('Precision:', round(precision,4))
print('Recall:', round(recall, 4))
print('F1-score:', round(f1, 4))

# Print the classification report
target_names = ['AmericanFootball', 'Basketball', 'BikeRacing', 'CarRacing', 'Fighting', 'Hockey', 'Soccer', 'TableTennis', 'Tennis', 'Volleyball']
print("\n------------------ Classification Report ------------------\n\n", classification_report(true_labels, pred_labels, target_names=target_names))

# Print the confusion matrix
print("------------------ Confusion Matrix ------------------\n\n",confusion_matrix(true_labels, pred_labels))

Accuracy: 0.764
Precision: 0.7794
Recall: 0.7557
F1-score: 0.7573

------------------ Classification Report ------------------

                   precision    recall  f1-score   support

AmericanFootball       0.84      0.77      0.80      2785
      Basketball       0.85      0.55      0.67      2649
      BikeRacing       0.80      0.69      0.74      2517
       CarRacing       0.80      0.78      0.79      1817
        Fighting       0.68      0.96      0.80      2856
          Hockey       0.77      0.97      0.86      3050
          Soccer       0.95      0.86      0.90      3272
     TableTennis       0.79      0.54      0.64      2169
          Tennis       0.60      0.67      0.63      3376
      Volleyball       0.72      0.76      0.74      2957

        accuracy                           0.76     27448
       macro avg       0.78      0.76      0.76     27448
    weighted avg       0.78      0.76      0.76     27448

------------------ Confusion Matrix ------------------

